In [20]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, transpile
from qiskit_aer import AerSimulator, Aer
from QArithmetic import add, cadd

# Registers and circuit.
a = QuantumRegister(4)
b = QuantumRegister(4)
anc = QuantumRegister(1)
three = QuantumRegister(4)
cb = ClassicalRegister(4)
c3 = ClassicalRegister(2)
qc = QuantumCircuit(a, b)
qc_new = QuantumCircuit(a, b, anc, three, cb, c3)

# init
qc_new.h(a[0]) # a = 0000 to 0111
qc_new.h(a[1])
qc_new.h(a[2])

qc_new.x(b[0]) # b = 0011
qc_new.x(b[1])

qc.barrier()

# If
add(qc, b, a, 3)

qc_new.append(qc, range(0, qc.num_qubits))

qc_new.cx(a[3], anc[0])

qc_new.append(qc.inverse(), range(0, qc.num_qubits))

# Encode 3
qc_new.cx(anc[0], three[0])
qc_new.cx(anc[0], three[1])

# Else
qc_new.x(anc[0])

# Cadd +1
qc_new.x(b[1]) # 0001
cadd(qc_new, anc[0], b, a, 3)

# Cond assignement
cadd(qc_new, anc[0], a, three, 3)

# Measure the results
qc_new.measure(three, cb)
# qc_new.measure(three, c3)


In [21]:
# Simulate the circuit.
simulator = AerSimulator()
qct = transpile(qc_new, simulator)
# print(qct)

result = Aer.get_backend('statevector_simulator').run(qct, shots=1024).result()
counts = result.get_counts()
print(counts)

{'00 0001': 130, '00 0010': 123, '00 0101': 109, '00 0100': 150, '00 0011': 512}
